In [1]:
import os
import pandas as pd
import re
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
import torch.nn.functional as F
import numpy as np

In [2]:

def ReadTxtIntoDataset(folder_path):
    folder_path = folder_path
    # 获取文件夹中所有文件的列表
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    x_list = []
    # 读取每个文件中的数据
    data_list = []
    for file_name in files:
        # print(file_name)
        file_path = os.path.join(folder_path, file_name)
        # 读取文本文件
        data = pd.read_csv(file_path,  delimiter=',', skipinitialspace=True, header=0)
        # 清理列名中的空格
        data.columns = data.columns.str.strip()
        x_values = (data['MISES'][72616],data['MISES'][178908].astype(float),data['MISES'][128977].astype(float)) #提取指定的地方做传感器的输入
        x_list.append(x_values)
        # print(data.head())
        # 提取第二列数据
        second_column = data['MISES'].astype(float).values
        # 打印提取的第二列数据
        # print("Second column values:", second_column)
        data_list.append(second_column)
    
    data = torch.tensor(x_list).float()  
    label =  torch.tensor(np.array(data_list)).float()

    # 打印划分后的数据
    return data,label

def mean_squared_error(y_true, y_pred):
    return torch.mean((y_true - y_pred)**2)

In [3]:

current_directory = os.getcwd()
print("当前工作目录：", current_directory)



当前工作目录： /root/autodl-tmp


In [4]:
folder_path = r"/root/autodl-fs/ODB-E"
data,label = ReadTxtIntoDataset(folder_path)

In [5]:
class My_dataset(Dataset):
    def __init__(self,data,labels):
        self.data = data
        self.labels = labels
           
    def __getitem__(self, idx):
        return self.data[idx],self.labels[idx]
    
    def __len__(self):
        return len(self.data)

In [6]:
train_data, val_data, train_labels, val_labels = train_test_split(data, label, test_size=0.2, random_state=42)
train_dataset = My_dataset(train_data, train_labels)
val_dataset = My_dataset(val_data, val_labels)

In [8]:
import torch
import torch.nn as nn
import numpy as np
x1 = torch.tensor([1.0,2.0,3.0])
x2 = torch.tensor([2.0,4.0,6.0])
loss = torch.mean(torch.abs(x1 - x2) / x1, axis=0)
# criterion = nn.MSELoss()  # 均方误差损失函数
# loss = criterion(x1, x2)
print(loss)
print(np.average(np.abs(x1 - x2) / x1, axis=0))

tensor(1.)
1.0


: 

In [7]:
class MLP(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_size,hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,output_size),
        )
    def forward(self,x):
        x = self.mlp(x)
        return x

In [8]:
input = 3
hidden = 2000
out = 245293
model = MLP(input, hidden, out)
batch_size = 16
learning_rate = 0.01
num_epochs = 100

In [9]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
# 定义损失函数和优化器
criterion = nn.MSELoss()  # 均方误差损失函数
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = model.to(device)


cuda


In [ ]:
num_epochs = 5
total_steps = len(train_dataloader)
val_loss_optimize = 10
for epoch in range(num_epochs):
    model.train()
    for i, (inputs, labels) in enumerate(train_dataloader):
        # 前向传播
        # print(labels)
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        # print(outputs.size())
        loss = criterion(outputs, labels)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        torch.cuda.empty_cache()
        # 打印训练信息
        if (i + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_steps}], Loss: {loss.item():.8f}')
            
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_data, batch_labels in val_dataloader:
            # 前向传播
            outputs = model(batch_data.to(device))
            loss = criterion(outputs, batch_labels.to(device))
            val_loss += loss.item()
            detach_outputs = outputs.detach()
            # 计算误差
        
    
    average_val_loss = val_loss / len(val_dataloader)
    
    print(f"Epoch {epoch+1}/{num_epochs}: Val Loss: {average_val_loss:.8f}")
    if average_val_loss < val_loss_optimize:
        torch.save(model.state_dict(), 'MLP_E_model_weights.pth202434')
        val_loss_optimize = average_val_loss

Epoch [1/5], Step [10/127], Loss: 0.00000016
Epoch [1/5], Step [20/127], Loss: 0.00000015
Epoch [1/5], Step [30/127], Loss: 0.00000030
Epoch [1/5], Step [40/127], Loss: 0.00000012
Epoch [1/5], Step [50/127], Loss: 0.00000027
Epoch [1/5], Step [60/127], Loss: 0.00000132
Epoch [1/5], Step [70/127], Loss: 0.00002089
Epoch [1/5], Step [80/127], Loss: 0.00032549
Epoch [1/5], Step [90/127], Loss: 0.00010376
